In [78]:
import requests
import pandas as pd
import yaml
from yaml import BaseLoader
from tqdm import tqdm
import json

# Load the YAML file.
with open("key.yaml") as f:
    key = yaml.load(f, Loader=BaseLoader)

In [80]:
df = pd.read_csv('./data/lasso-sneaks-cleaned.csv')
df

,Episode String,Season,Episode,Shoe Name,Shoe Thumbnail,id
0,season-1-episode-2,1,2,Jordan Zoom Zero Gravity,https://shoesoflasso.com/wp-content/uploads/20...,shoe_02
1,season-1-episode-2,1,2,Jordan React Havoc Paris Saint-Germain (PSG),https://shoesoflasso.com/wp-content/uploads/20...,shoe_03
2,season-1-episode-3,1,3,Nike Tanjun in Midnight Navy/Game Royal/White,https://shoesoflasso.com/wp-content/uploads/20...,shoe_04
3,season-1-episode-5,1,5,Nike Air Huarache Premium ‘Varsity Jacket’ in Red,https://shoesoflasso.com/wp-content/uploads/20...,shoe_05
4,season-1-episode-6,1,6,Odyssey React 2 Flyknit ‘Black Racer Blue’,https://shoesoflasso.com/wp-content/uploads/20...,shoe_07
5,season-1-episode-8,1,8,Eric Koston Air Jordan 1 Low ‘Powder Blue’,https://shoesoflasso.com/wp-content/uploads/20...,shoe_09
6,season-1-episode-10,1,10,JIMMY CHOO Diamond X Strap/F Leather Sneakers,https://shoesoflasso.com/wp-content/uploads/20...,shoe_11
7,season-1-episode-10,1,10,Nike Blazer Low X ‘White Light Photo Blue’,https://shoesoflasso.com/wp-content/uploads/20...,shoe_12
8,season-1-episode-10,1,10,Eric Koston x Air Jordan 1 Low SB ‘Powder Blue’,https://shoesoflasso.com/wp-content/uploads/20...,shoe_13
9,season-1-episode-10,1,10,Nike Waffle One Summer of Sports Pack,https://shoesoflasso.com/wp-content/uploads/20...,shoe_14


In [81]:
shoes = list(df['Shoe Name'].unique())
shoes

['Jordan Zoom Zero Gravity',
 'Jordan React Havoc Paris Saint-Germain (PSG)',
 'Nike Tanjun in Midnight Navy/Game Royal/White',
 'Nike Air Huarache Premium ‘Varsity Jacket’ in Red',
 'Odyssey React 2 Flyknit ‘Black Racer Blue’',
 'Eric Koston Air Jordan 1 Low ‘Powder Blue’',
 'JIMMY CHOO Diamond X Strap/F Leather Sneakers',
 'Nike Blazer Low X ‘White Light Photo Blue’',
 'Eric Koston x Air Jordan 1 Low SB ‘Powder Blue’',
 'Nike Waffle One Summer of Sports Pack',
 'ASIC Mexico 66 ‘Kill Bill’',
 'Nike Air Jordan 3 ‘Muslin’',
 'Red Nike Daybreak Type ‘Red Racer Blue’',
 'Nike Daybreak ‘Light Armory Blue’',
 'Nike Air Jordan 5 “Chinese New Year”',
 'Nike Air Jordan 1 Low Light Smoke Grey',
 'Nike Air Jordan 1 Low ‘White Camo’',
 'Air Jordan 13 “Lucky Green”',
 'Nike Phantom Vision Academy Indoor Turf Soccer Shoes',
 'New Balance 515 Sport V2 White Pigment Red ML515OTX',
 'Nike Jordan Delta Breathe',
 'Nike Air Jordan 1 Retro High OG ‘University Blue’',
 'Nike Jordan 1 Low Mocha',
 'Nike Ai

In [82]:
import re
import time

url = "https://sneaker-database-stockx.p.rapidapi.com/getproducts"

headers = {
	"X-RapidAPI-Key": key['RAPID_API_KEY'],
	"X-RapidAPI-Host": "sneaker-database-stockx.p.rapidapi.com"
}

sneaker_dict = {}
for i, sneaker in enumerate(shoes): 
    
	try:
		# cleaning up quote characters
		sneaker_str = re.sub(r'[^A-Za-z0-9 ]+', ' ', sneaker)
		sneaker_str = re.sub(' +', ' ', sneaker_str).strip()
		sneaker_str = sneaker_str.replace(" Light ", " ")
		sneaker_str = sneaker_str.replace(" X Strap F Leather ", " ")

		# make API request to the getProducts endpoint of the Sneaker Database - StockX API, save response in dict
		querystring = {"keywords":sneaker_str,"limit":"1"}
		response = requests.get(url, headers=headers, params=querystring)
		sneaker_dict[sneaker] = json.loads(response.text)[0]
		print(i, f"SUCCESS: {sneaker}")

	except Exception as e:
		print(i, f"ERROR: {sneaker} ({sneaker_str})")
		print(e)

	# wait 2 minutes just in case we hit the rate limit
	time.sleep(120)


0 SUCCESS: Jordan Zoom Zero Gravity
1 SUCCESS: Jordan React Havoc Paris Saint-Germain (PSG)
2 SUCCESS: Nike Tanjun in Midnight Navy/Game Royal/White
3 SUCCESS: Nike Air Huarache Premium ‘Varsity Jacket’ in Red
4 SUCCESS: Odyssey React 2 Flyknit ‘Black Racer Blue’
5 SUCCESS: Eric Koston Air Jordan 1 Low ‘Powder Blue’
6 SUCCESS: JIMMY CHOO Diamond X Strap/F Leather Sneakers
7 SUCCESS: Nike Blazer Low X ‘White Light Photo Blue’
8 SUCCESS: Eric Koston x Air Jordan 1 Low SB ‘Powder Blue’
9 SUCCESS: Nike Waffle One Summer of Sports Pack
10 SUCCESS: ASIC Mexico 66 ‘Kill Bill’
11 SUCCESS: Nike Air Jordan 3 ‘Muslin’
12 ERROR: Red Nike Daybreak Type ‘Red Racer Blue’ (Red Nike Daybreak Type Red Racer Blue)
'NoneType' object is not subscriptable
13 ERROR: Nike Daybreak ‘Light Armory Blue’ (Nike Daybreak Armory Blue)
Expecting value: line 1 column 1 (char 0)
14 SUCCESS: Nike Air Jordan 5 “Chinese New Year”
15 SUCCESS: Nike Air Jordan 1 Low Light Smoke Grey
16 SUCCESS: Nike Air Jordan 1 Low ‘White C

In [83]:
with open("./data/sneaker_info.json", "w") as outfile:
    json.dump(sneaker_dict, outfile)